In [1]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [2]:
%load_ext autoreload

%autoreload 2

In [5]:
import torch
from glasses.nn.models.segmentation.unet import UNet, UNetEncoder
from glasses.nn.models.classification.senet import SENetBasicBlock
from glasses.nn.models.classification.resnet import ResNetEncoder, ResNet
from glasses.nn.blocks import Conv2dPad
from functools import partial
x = torch.rand((1, 1, 32 * 12, 32*12))
# custom encoder
resnet = ResNet.resnet18(in_channels=1)
# we need to change the first conv in order to accept a gray image
# resnet.encoder.blocks[0].block[0].block.block.conv1 = Conv2dPad(1, 64, kernel_size=1)
# unet = UNet(1, n_classes=2, widths=resnet.encoder.widths)
# use resnet18 as encoder
resnet = ResNet.resnet18(in_channels=1)
unet = UNet(1, n_classes=2, widths=resnet.encoder.widths)
unet.encoder = resnet.encoder
unet(x)

unet = UNet(encoder=partial(UNetEncoder, block=SENetBasicBlock))
unet(x)

encoder = ResNetEncoder(in_channels=1, widths=[64,128,256,512,1024], depths=[2,2,2,2,2])
unet = UNet(1, n_classes=2, widths=encoder.widths)
unet.encoder = encoder
unet(x)
# # custom block
# unet = UNet(down_block=SENetBasicBlock, up_block=SENetBasicBlock)
# unet(x)


tensor([[[[-0.5097, -0.1584, -0.1039,  ..., -0.4699, -0.4895, -0.3134],
          [-0.3268, -0.0977,  0.5717,  ...,  0.1253, -0.2982,  0.0250],
          [-0.3186,  0.0352,  0.6240,  ..., -0.3029, -0.3958, -0.6869],
          ...,
          [-0.4024,  0.4335,  0.2689,  ..., -0.9266, -0.0629, -0.7173],
          [ 0.0423, -0.1966,  0.1985,  ...,  0.1283,  0.2436,  0.0872],
          [-0.1668, -0.1652,  0.2772,  ..., -0.2744, -0.3322, -0.2125]],

         [[-0.2416, -0.1410, -0.3617,  ..., -0.4207, -0.4389,  0.0384],
          [-0.5749, -0.6739, -0.3115,  ..., -0.2171, -0.1255,  0.0023],
          [-0.1302, -0.2923, -0.1579,  ..., -0.1351, -0.0416, -0.3534],
          ...,
          [-0.0748, -0.3791,  0.0713,  ..., -0.2042, -0.3289,  0.3153],
          [-0.4919, -0.7480, -0.2047,  ..., -0.1089,  0.3773, -0.2863],
          [ 0.0320, -0.0969,  0.0226,  ...,  0.3943,  0.4132, -0.2187]]]],
       grad_fn=<MkldnnConvolutionBackward>)

In [ ]:
unet

In [ ]:
resnet.encoder

In [ ]:
from __future__ import annotations
import torch
import torch.nn.functional as F
from torch import nn
from torch import Tensor
from collections import OrderedDict
from typing import List
from functools import partial
from glasses.nn.blocks import ConvBnAct, Conv2dPad
from glasses.nn.models.segmentation.unet import UNet


In [ ]:
from glasses.nn.models.classification.efficientnet import EfficientNet, MemoryEfficientSwish
from torchsummary import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_dst  = EfficientNet(activation=MemoryEfficientSwish)

summary(model_dst.to(device), (3,224,224))

model_dst

In [ ]:
from dataclasses import dataclass
import numpy as np

@dataclass
class CompoundScaling:
    alpha: float = 1.2
    beta: float = 1.1
    gamma: float = 1.15
    
    def __call__(self, model, depths, width):
        depths = np.array(depths) * self.beta
        width = np.array(width) * self.alpha
        depths = np.ceil(depths)
        width = np.ceil(width)
        
        return depths, width
        
CompoundScaling()(EfficientNet, [1, 2, 2, 3, 3, 4, 1], [32, 16, 24, 40, 80, 112, 192, 320, 1280])

In [ ]:
224 * 1.15

In [ ]:
model_dst

In [ ]:
y_dst

tensor([[-9.3545e-01,  4.4536e-01,  4.2809e-02, -7.1374e-01,  3.6706e-01,
          2.3940e-01, -1.3259e+00, -3.2074e-01, -1.6859e+00, -7.2495e-01,
         -1.2006e+00, -5.1331e-01, -7.6875e-01, -1.4668e+00, -8.2695e-01,
         -4.0084e-02,  3.0943e-01, -7.2798e-01, -4.7880e-01, -8.8295e-01,
         -1.6215e+00,  7.9493e-01,  3.6127e-02, -5.1066e-01, -4.0636e-01,
         -8.5400e-01, -1.2117e+00, -2.1563e-01, -1.1900e+00, -5.3588e-01,
         -9.6766e-01, -8.8796e-01, -1.2135e+00, -6.0439e-01, -7.0122e-01,
         -1.4644e+00, -1.1432e+00, -9.4588e-01, -9.1610e-01, -1.1081e+00,
         -7.2389e-01, -8.2417e-01, -8.6236e-01, -1.3655e+00, -1.6005e+00,
         -1.3012e-01, -1.6388e+00, -7.9500e-01, -1.2640e+00, -1.1749e+00,
         -1.2268e+00, -4.6094e-01, -7.8901e-01, -1.4785e+00, -2.4503e+00,
         -5.8356e-01, -1.4344e+00, -6.2406e-01, -1.2202e+00, -3.8103e-01,
         -1.2667e+00, -7.2553e-01, -1.5403e+00, -2.9658e-01, -7.7501e-01,
         -4.5203e-01, -1.2975e+00, -1.

In [ ]:
model_src